In [7]:
import os, re, csv
import pandas as pd
import numpy as np

In [36]:
eta_df = pd.read_csv("../자료/질문2.csv", encoding='ANSI')
eta_df.head(10)

eta_df = eta_df.replace(np.nan, "['']", regex=True)

eta_df.head(10)

,Unnamed: 0,내용,댓글,댓글쓴이,대댓글여부,id
0,0,안녕하세요 조교님! 혹시 부전공 신청은 막학기에 불가능한가요??ㅜ이번학기가 막학기고...,[''],[''],[''],112883775
1,1,"안녕하세요, 조교님!저는 전역이 5주 정도 남은 군인입니다. 전역 후 계획을 세우다...",[''],[],[''],112880808
2,2,방학중 어학연수에 대해 질문이 있어서요!휴학중 학점인정은 재학 기간중 1회에 한해서...,[''],[],[''],112864005
3,3,안녕하세요 조교님!제가지금 4학년 1학기 인데4학년은 1학기에 12학점 미만으로 학...,[''],[],[''],112848838
4,4,조교님 안녕하세요!재수강을 하려고 하는데재수강과목선택란에 조회결과가 없다고 뜨면다시...,[''],[],[''],112813045
5,5,안녕하세요 조교님!!자퇴를 하려고 하는데 어디서 신청하면 되나요??,"['종합정보시스템 > 학적변동신청 > 자퇴신청을 한 다음, 자퇴원서를 작성을 해서 ...",['에타조교'],['parent'],112748915
6,6,안녕하세요 조교님!1/4학기 이전에 휴학 신청을 하면 등록금 환불이라고 들었는데 맞...,['1. 등록금 환불은 자퇴 시에만 가능합니다 휴학을 하면 등록휴학으로 처리됩니다 ...,"['에타조교', '익명(글쓴이)', '에타조교', '익명(글쓴이)']","['parent', 'child', 'child', 'child']",112663087
7,7,안녕하세요 조교님!!ㅠㅠ제가 항상 동일과목을 재수강 해서 자동연결이 됐어서이번에도 ...,"['음 지금 이게 전공인지 교양인지 또 (1),(2)로 나뉘어진 연강수업인지 아니면...","['에타조교', '익명(글쓴이)', '에타조교', '익명(글쓴이)']","['parent', 'child', 'child', 'child']",112627947
8,8,"안녕하세요 조교님!이번에 인성을 위한 lectio(언어와 문학, p/f 과목)을 듣...",['미네르바는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동...,['에타조교'],['parent'],112619096
9,9,안녕하세요 조교님! 제가 이번학기에 수강하는 언어와 문학 과목을 재수강 연결 하려고...,['대학외국어는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (...,['에타조교'],['parent'],112563109


In [13]:
content = eta_df['내용'].tolist()
comments = eta_df['댓글'].tolist()
writer = eta_df['댓글쓴이'].tolist()
ids = eta_df['id'].tolist()

In [14]:
comments[:10]

["['']",
 "['']",
 "['']",
 "['']",
 "['']",
 "['종합정보시스템 > 학적변동신청 > 자퇴신청을 한 다음, 자퇴원서를 작성을 해서 학종지에 제출하면 됩니다']",
 "['1. 등록금 환불은 자퇴 시에만 가능합니다 휴학을 하면 등록휴학으로 처리됩니다 2. 지금 코로나 때문에 학사일정이 다소 바뀌었는데 학기 1/4선 계산 등은 학종지에 직접 문의하셔서 확인바랍니다', '현재 해외에 있어서 학종지에 직접 문의를 하기가 힘든 상황입니다', '그런 경우엔 보통 지인이나 가족을 통해서 문의를 합니다 저도 예전엔 그랬고요..', '아 넵 알겠습니다']",
 "['음 지금 이게 전공인지 교양인지 또 (1),(2)로 나뉘어진 연강수업인지 아니면 학기 상관없는 동일수업인지 알 수가 없어서 답변이 제한되네요...우선 종정시에 접속하셔서 수업/수강관리 > 재수강과목선택 부분을 확인바랍니다', '전공수업에 1,2 나눠지지 않은 동일과목입니다! 자동연결될줄 알았는데 학교에서 미연결과목있다고 문자오고 재수강과목선택 들어가서 성적연결신청을 누르면 이전성적 조회결과가 없다고 뜹니다ㅠㅠ 교수님까지 똑같은 강의인데ㅠㅠ 자동연결이 되는걸까요ㅠㅠ', '그럼 자동연결이 되었을텐데 이상하네요 혹시 전공 말고 교양 등 다른 과목이 아닌지 확인바라며 만일 그것도 아니라면 학종지에 문의바랍니다', '원래 재수강 연결할때 이전성적이 뜨고 연결할수있는건가요? 전 아무리봐도 같은 과목인데 자동연결이라고 안써져있고..ㅠㅠ']",
 "['미네르바는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일과목으로만 재수강 가능)']",
 "['대학외국어는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일과목으로만 재수강 가능)']"]

In [27]:
#리스트로 변환 방식 예시
a = "['1. 등록금 환불은 자퇴 시에만 가능합니다 휴학을 하면 등록휴학으로 처리됩니다 2. 지금 코로나 때문에 학사일정이 다소 바뀌었는데 학기 1/4선 계산 등은 학종지에 직접 문의하셔서 확인바랍니다', '현재 해외에 있어서 학종지에 직접 문의를 하기가 힘든 상황입니다', '그런 경우엔 보통 지인이나 가족을 통해서 문의를 합니다 저도 예전엔 그랬고요..', '아 넵 알겠습니다']"
a =re.sub("^\[[\"\']", '', a)#양 끝의 "[", "]" 제거
a =re.sub("[\"\']\]$", '', a)
test =re.split("[\"\'], [\"\']", a) # ', "   ", '  ', ' 등을 기준으로 split하여 리스트로 만든다. 
test

['1. 등록금 환불은 자퇴 시에만 가능합니다 휴학을 하면 등록휴학으로 처리됩니다 2. 지금 코로나 때문에 학사일정이 다소 바뀌었는데 학기 1/4선 계산 등은 학종지에 직접 문의하셔서 확인바랍니다',
 '현재 해외에 있어서 학종지에 직접 문의를 하기가 힘든 상황입니다',
 '그런 경우엔 보통 지인이나 가족을 통해서 문의를 합니다 저도 예전엔 그랬고요..',
 '아 넵 알겠습니다']

In [28]:
def string_to_list(str_ls):
    res = []
    for s in str_ls:
        temp = re.sub("^\[[\"\']", '', s)
        temp = re.sub("[\"\']\]$", '', temp)
        res.append(re.split("[\"\'], [\"\']", temp))
    return res

In [38]:
comments_ls = string_to_list(comments)
writer_ls = string_to_list(writer)
print(comments_ls[:10])
# print(writer_ls[:10])

[[''], [''], [''], [''], [''], ['종합정보시스템 > 학적변동신청 > 자퇴신청을 한 다음, 자퇴원서를 작성을 해서 학종지에 제출하면 됩니다'], ['1. 등록금 환불은 자퇴 시에만 가능합니다 휴학을 하면 등록휴학으로 처리됩니다 2. 지금 코로나 때문에 학사일정이 다소 바뀌었는데 학기 1/4선 계산 등은 학종지에 직접 문의하셔서 확인바랍니다', '현재 해외에 있어서 학종지에 직접 문의를 하기가 힘든 상황입니다', '그런 경우엔 보통 지인이나 가족을 통해서 문의를 합니다 저도 예전엔 그랬고요..', '아 넵 알겠습니다'], ['음 지금 이게 전공인지 교양인지 또 (1),(2)로 나뉘어진 연강수업인지 아니면 학기 상관없는 동일수업인지 알 수가 없어서 답변이 제한되네요...우선 종정시에 접속하셔서 수업/수강관리 > 재수강과목선택 부분을 확인바랍니다', '전공수업에 1,2 나눠지지 않은 동일과목입니다! 자동연결될줄 알았는데 학교에서 미연결과목있다고 문자오고 재수강과목선택 들어가서 성적연결신청을 누르면 이전성적 조회결과가 없다고 뜹니다ㅠㅠ 교수님까지 똑같은 강의인데ㅠㅠ 자동연결이 되는걸까요ㅠㅠ', '그럼 자동연결이 되었을텐데 이상하네요 혹시 전공 말고 교양 등 다른 과목이 아닌지 확인바라며 만일 그것도 아니라면 학종지에 문의바랍니다', '원래 재수강 연결할때 이전성적이 뜨고 연결할수있는건가요? 전 아무리봐도 같은 과목인데 자동연결이라고 안써져있고..ㅠㅠ'], ['미네르바는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일과목으로만 재수강 가능)'], ['대학외국어는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일과목으로만 재수강 가능)']]


In [41]:
#에타 조교의 첫번째 답변만 추출, 댓글쓴이와 댓글은 그 개수가 1대1 대응
answer = []
found = 0 #에타조교를 찾았는지 여부
for i, c in enumerate(comments_ls):
    found = 0 #찾았음 표시 토글 초기화
    for j, w in enumerate(writer_ls[i]):
        if w=="에타조교": #에타조교가 가장 처음으로 쓴 댓글을 만나면, 
            answer.append(c[j]) #현재 comment들 중 에타조교가 쓴 글의 인덱스를 answer에 추가
            found = 1 #찾았음을 표시
            break
    #에타조교가 쓴 글이 없으면,
    if found == 0:
        answer.append("no answer by 에타조교")
print(len(answer), len(ids)) #개수가 일치하는 지 확인
answer[:10]


3024 3024


['no answer by 에타조교',
 'no answer by 에타조교',
 'no answer by 에타조교',
 'no answer by 에타조교',
 'no answer by 에타조교',
 '종합정보시스템 > 학적변동신청 > 자퇴신청을 한 다음, 자퇴원서를 작성을 해서 학종지에 제출하면 됩니다',
 '1. 등록금 환불은 자퇴 시에만 가능합니다 휴학을 하면 등록휴학으로 처리됩니다 2. 지금 코로나 때문에 학사일정이 다소 바뀌었는데 학기 1/4선 계산 등은 학종지에 직접 문의하셔서 확인바랍니다',
 '음 지금 이게 전공인지 교양인지 또 (1),(2)로 나뉘어진 연강수업인지 아니면 학기 상관없는 동일수업인지 알 수가 없어서 답변이 제한되네요...우선 종정시에 접속하셔서 수업/수강관리 > 재수강과목선택 부분을 확인바랍니다',
 '미네르바는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일과목으로만 재수강 가능)',
 '대학외국어는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일과목으로만 재수강 가능)']

In [46]:
raw_data = {'질문': content, 
            '답변': answer, 
            'id': ids}
data = pd.DataFrame(raw_data)
data[:10]

,질문,답변,id
0,안녕하세요 조교님! 혹시 부전공 신청은 막학기에 불가능한가요??ㅜ이번학기가 막학기고...,no answer by 에타조교,112883775
1,"안녕하세요, 조교님!저는 전역이 5주 정도 남은 군인입니다. 전역 후 계획을 세우다...",no answer by 에타조교,112880808
2,방학중 어학연수에 대해 질문이 있어서요!휴학중 학점인정은 재학 기간중 1회에 한해서...,no answer by 에타조교,112864005
3,안녕하세요 조교님!제가지금 4학년 1학기 인데4학년은 1학기에 12학점 미만으로 학...,no answer by 에타조교,112848838
4,조교님 안녕하세요!재수강을 하려고 하는데재수강과목선택란에 조회결과가 없다고 뜨면다시...,no answer by 에타조교,112813045
5,안녕하세요 조교님!!자퇴를 하려고 하는데 어디서 신청하면 되나요??,"종합정보시스템 > 학적변동신청 > 자퇴신청을 한 다음, 자퇴원서를 작성을 해서 학종...",112748915
6,안녕하세요 조교님!1/4학기 이전에 휴학 신청을 하면 등록금 환불이라고 들었는데 맞...,1. 등록금 환불은 자퇴 시에만 가능합니다 휴학을 하면 등록휴학으로 처리됩니다 2....,112663087
7,안녕하세요 조교님!!ㅠㅠ제가 항상 동일과목을 재수강 해서 자동연결이 됐어서이번에도 ...,"음 지금 이게 전공인지 교양인지 또 (1),(2)로 나뉘어진 연강수업인지 아니면 학...",112627947
8,"안녕하세요 조교님!이번에 인성을 위한 lectio(언어와 문학, p/f 과목)을 듣...",미네르바는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일과...,112619096
9,안녕하세요 조교님! 제가 이번학기에 수강하는 언어와 문학 과목을 재수강 연결 하려고...,대학외국어는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일...,112563109


In [49]:
data.to_csv("../자료/에타조교_정답추출.csv", mode='w',index=False, encoding='utf-8')